# Importing Required Libraries

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Read the text file
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

tokenize the text to create a sequence of words:

In [2]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

create input-output pairs by splitting the text into sequences of tokens and forming n-grams from the sequences:

In [3]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [4]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [5]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [6]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [7]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2,208,800
Trainable params: 2,208,800
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 109s 36ms/step - loss: 6.2294 - accuracy: 0.0772
Epoch 2/100
3010/3010 [==============================] - 100s 33ms/step - loss: 5.4945 - accuracy: 0.1259
Epoch 3/100
3010/3010 [==============================] - 104s 34ms/step - loss: 5.1186 - accuracy: 0.1482
Epoch 4/100
3010/3010 [==============================] - 109s 36ms/step - loss: 4.7954 - accuracy: 0.1652
Epoch 5/100
3010/3010 [==============================] - 105s 35ms/step - loss: 4.4909 - accuracy: 0.1807
Epoch 6/100
3010/3010 [==============================] - 106s 35ms/step - loss: 4.2025 - accuracy: 0.2017
Epoch 7/100
3010/3010 [==============================] - 106s 35ms/step - loss: 3.9266 - accuracy: 0.2262
Epoch 8/100
3010/3010 [==============================] - 108s 36ms/step - loss: 3.6698 - accuracy: 0.2584
Epoch 9/100
3010/3010 [==============================] - 106s 35ms/step - loss: 3.4270 - accuracy: 0.2909
Epoch 10/100
3010/3010 [======================

In [9]:
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 21ms/step
I will leave if they like nothing of


# Summary
Next word prediction is a language modelling task in Machine Learning that aims to predict the most probable word or sequence of words that follows a given input context. This task utilizes statistical patterns and linguistic structures to generate accurate predictions based on the context provided. 